# Live plotting for LiberTEM UDFs

This example requires additional packages `ipympl` and `ipywidgets` for arranging plots.

See also https://ipywidgets.readthedocs.io/en/stable/user_install.html#installing-in-classic-jupyter-notebook
in case the plots don't show.

In [1]:
%matplotlib widget

In [2]:
%load_ext autoreload

In [3]:
%load_ext line_profiler

In [4]:
import time

import matplotlib.pyplot as plt
import ipywidgets
import numpy as np

import libertem.api as lt
from libertem.viz.mpl import MPLLive2DPlot
from libertem.viz.bqp import BQLive2DPlot

In [5]:
%autoreload
ctx = lt.Context(plot_class=BQLive2DPlot)

c:\users\dwebe\documents\src\libertem\src\libertem\utils\devices.py:72: RuntimeWarning: CUDARuntimeError('cudaErrorUnsupportedPtxVersion: the provided PTX was compiled with an unsupported toolchain.')
  warnings.warn(repr(e), RuntimeWarning)


## Specifying the dataset

Most formats can be loaded using the `"auto"` type, but some may need additional parameters.

See the [loading data](https://libertem.github.io/LiberTEM/formats.html) section of the LiberTEM docs for details.

In [6]:
ds = ctx.load("auto", path="E:/LargeData/LargeData/ER-C-1/projects/ptycho-4.0/data/live-ssb-paper/Ptycho01/20200518 165148/default.hdr")
ds.set_num_cores(32)

After loading, some information is available in the `diagnostics` attribute:

In [7]:
ds.diagnostics

[{'name': 'Data type', 'value': 'u08'},
 {'name': 'Partition shape', 'value': '(512, 256, 256)'},
 {'name': 'Number of partitions', 'value': '32'},
 {'name': 'Number of frames skipped at the beginning', 'value': 0},
 {'name': 'Number of frames ignored at the end', 'value': 0},
 {'name': 'Number of blank frames inserted at the beginning', 'value': 0},
 {'name': 'Number of blank frames inserted at the end', 'value': 0}]

In [8]:
from libertem.udf import UDF


class SumOfPixels(UDF):
    def get_result_buffers(self):
        return {
            'sum_of_pixels': self.buffer(kind='nav', dtype='float32')
        }
    
    def process_frame(self, frame):
        self.results.sum_of_pixels[:] = np.sum(frame)

This can now be run using the `Context.run_udf` method:

In [9]:
udf = SumOfPixels()

In [10]:
roi = np.ones(ds.shape.nav, dtype=bool)
roi[:, :40] = 0

In [11]:
ctx.plot_class = MPLLive2DPlot

In [12]:
%autoreload
res_pixelsum = ctx.run_udf(dataset=ds, udf=udf, roi=roi, plots=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%autoreload
res_pixelsum = ctx.run_udf(dataset=ds, udf=udf, roi=roi, plots=[['sum_of_pixels', ('sum_of_pixels', np.sqrt)]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
def my_custom_channel(udf_result, damage):
    return (udf_result['sum_of_pixels'].data > 9.1e4, damage)

def square(buffer):
    return buffer**2

In [15]:
%autoreload
live_plot = MPLLive2DPlot(dataset=ds, udf=udf, channel='sum_of_pixels')
live_plot_2 = MPLLive2DPlot(dataset=ds, udf=udf, channel=my_custom_channel)
live_plot_3 = MPLLive2DPlot(dataset=ds, udf=udf, channel=('sum_of_pixels', square))

live_bqp = BQLive2DPlot(dataset=ds, udf=udf, channel='sum_of_pixels')
live_bqp_2 = BQLive2DPlot(dataset=ds, udf=udf, channel=my_custom_channel)
live_bqp_3 = BQLive2DPlot(dataset=ds, udf=udf, channel=('sum_of_pixels', square), title="Hello")

In [16]:
outputs = []

for plot in (live_plot, live_plot_2, live_plot_3):
    output = ipywidgets.Output()
    with output:
        plot.display()
        plot.fig.tight_layout()
        plot.fig.set_size_inches((3, 3))
        plot.fig.canvas.toolbar_position = 'bottom'
    outputs.append(output)

l1 = ipywidgets.HBox(outputs)

outputs2 = []
for plot in (live_bqp, live_bqp_2, live_bqp_3):
    output = ipywidgets.Output()
    with output:
        plot.display()
    outputs2.append(output)
l2 = ipywidgets.HBox(outputs2)
ipywidgets.VBox((l1, l2))

In [17]:
%%time
ctx.run_udf(dataset=ds, udf=udf, plots=[live_plot, live_plot_2, live_plot_3, live_bqp, live_bqp_2, live_bqp_3])

Wall time: 2.36 s


{'sum_of_pixels': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}

In [18]:
from libertem.udf.raw import PickUDF

In [19]:
roi = np.zeros(ds.shape.nav, dtype=bool)
roi[0,0] = True
pick_udf = PickUDF()

In [20]:
%autoreload
ctx.run_udf(dataset=ds, udf=pick_udf, roi=roi, plots=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'intensity': <BufferWrapper kind=single dtype=uint8 extra_shape=(1, 256, 256)>}

In [21]:
def custom_single_channel(udf_result, damage):
    return (udf_result['intensity'].data.squeeze().astype(float)**2, True)

In [22]:
%autoreload
live_plot_pick = MPLLive2DPlot(dataset=ds, udf=pick_udf, channel=custom_single_channel, roi=roi)
live_plot_pick.display()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
%autoreload
ctx.run_udf(dataset=ds, udf=pick_udf, roi=roi, plots=[live_plot_pick])

{'intensity': <BufferWrapper kind=single dtype=uint8 extra_shape=(1, 256, 256)>}